# Subword Tokenizer

| **Tokenizer 방식** | **토큰 단위**                      | **vocab size** | **미등록 단어에 대한 가정**                                                                                  |
|---------------------|------------------------------------|----------------|-------------------------------------------------------------------------------------------------------------|
| **사전 기반**       | 알려진 단어/형태소의 결합           | unlimited       | - 알려진 단어/형태소의 결합이라고 가정<br>- 필요한 형태소 분석 가능<br>- 사전에 등록되지 않은 단어는 UNK 처리 |
| **sub-word**        | 알려진 글자 및 sub-word            | fixed           | - 알려진 sub-words로 분해<br>- 예: appear → app + ear<br>- 자주 등장하는 단어를 제대로 인식 가능<br>- UNK의 개수 최소화 |